## Imports

In [1]:
import findspark
findspark.init()
from pyspark.context import SparkContext, SparkConf
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, ArrayType, StringType, ShortType, FloatType
from pyspark.sql import SparkSession
from spark_functions import *


c:\Users\39392\anaconda3\envs\spark_env\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
sc = SparkContext().getOrCreate()

In [3]:
spark = SparkSession(sc).builder \
    .appName("dica33_doctors") \
    .getOrCreate()
    #.master("local[1]") \
print(spark)

## Mapping

In [4]:
category_mapping = {
    'Allergologia e immunologia clinica': 'Allergologia e immunologia',
    'Anatomia patologica': 'Anatomia patologica',
    'Andrologia': 'Andrologia',
    'Anestesia e rianimazione': 'Anestesia e rianimazione',
    'Audiologia e Foniatria': 'Audiologia e foniatria',
    'Biochimica clinica': 'Biochimica clinica',
    'Biometria e statistica medica': 'Biometria e statistica medica',
    'Cardiochirurgia': 'Cardiochirurgia',
    'Cardiologia': 'Cardiologia',
    'Chirurgia apparato digerente': "Chirurgia dell'apparato digerente",
    "Chirurgia d'urgenza": 'Chirurgia di urgenza e Pronto Soccorso',
    'Chirurgia generale': 'Chirurgia generale',
    'Chirurgia maxillo facciale': 'Chirurgia maxillo-facciale',
    'Chirurgia pediatrica': 'Chirurgia pediatrica',
    'Chirurgia plastica ricostruttiva': 'Chirurgia plastica e ricostruttiva',
    'Chirurgia toracica': 'Chirurgia toracica',
    'Chirurgia vascolare, angiologia': 'Chirurgia vascolare e angiologia',
    'Dermatologia e venereologia': 'Dermatologia e venereologia',
    'Ematologia': 'Ematologia',
    'Endocrinologia, Diabetologia e malattie del ricambio': 'Diabetologia e malattie del metabolismo',
    'Epidemiologia': 'Epidemiologia',
    'Farmacologia': 'Farmacologia',
    'Fisiatria': 'Fisiatria',
    'Fisiologia': 'Fisiologia',
    'Gastroenterologia': 'Gastroenterologia e endoscopia digestiva',
    'Genetica medica': 'Genetica medica',
    'Geriatria': 'Geriatria',
    'Ginecologia e ostetricia': 'Ginecologia e ostetricia',
    'Idrologia medica': 'Idrologia medica',
    'Igiene e medicina preventiva': 'Igiene e medicina preventiva',
    'Malattie apparato respiratorio': 'Pneumologia',
    'Malattie del fegato e del ricambio': 'Diabetologia e malattie del metabolismo',
    'Malattie infettive': 'Malattie infettive',
    'Medicina aeronautica e spaziale': 'Medicina aeronautica e spaziale',
    'Medicina alternativa (omeopatia,agopuntura...)': 'Medicina alternativa',
    "Medicina d'emergenza-urgenza": "Medicina d'urgenza",
    'Medicina del lavoro': 'Medicina del lavoro',
    'Medicina dello sport': 'Medicina dello sport',
    'Medicina estetica': 'Medicina estetica',
    'Medicina fisica e riabilitazione': 'Medicina fisica e riabilitativa',
    'Medicina generale': 'Medicina generale',
    'Medicina interna': 'Medicina interna',
    'Medicina laboratoristica': 'Medicina di laboratorio',
    'Medicina legale': 'Medicina legale e delle assicurazioni',
    'Medicina nucleare': 'Medicina nucleare',
    'Medicina tropicale': 'Medicina tropicale',
    'Microbiologia e virologia': 'Microbiologia e virologia',
    'Nefrologia': 'Nefrologia',
    'Neurochirurgia': 'Neurochirurgia',
    'Neurofisiopatologia': 'Neurofisiopatologia',
    'Neurologia': 'Neurologia',
    'Neuropsichiatria infantile': 'Neuropsichiatria infantile',
    'Odontoiatria': 'Odontoiatria e odontostomatologia',
    'Oftalmologia': 'Oftalmologia',
    'Oncologia': 'Oncologia medica',
    'Ortopedia e traumatologia': 'Ortopedia',
    'Otorinolaringoiatria': 'Otorinolaringoiatria',
    'Patologia clinica': 'Patologia clinica',
    'Pediatria': 'Pediatria',
    'Psichiatria': 'Psichiatria',
    'Psicologia': 'Psicologia',
    'Psicologia clinica': 'Psicologia Clinica',
    'Radiodiagnostica': 'Radiologia',
    'Radioterapia': 'Radioterapia',
    'Reumatologia': 'Reumatologia',
    "Scienza dell'alimentazione": "Scienza dell'alimentazione",
    'Stomatologia': 'Stomatologia',
    'Tossicologia medica': 'Tossicologia Medica',
    'Urologia': 'Urologia',
    'Valutazione psicologica e consulenza (counselling)': 'Psicoterapia'
}

mapping_udf = udf(lambda x: map_category(x, category_mapping), StringType())

## Caricamento 

In [7]:
dica33_doctors_df = load_dataframe(spark, '../dica33_doctors/data')
dica33_doctors_df.select('*').where(col('specialization').isNull()).show()

../dica33_doctors/data loaded
root
 |-- URL: string (nullable = true)
 |-- name: string (nullable = true)
 |-- specialization: string (nullable = true)
 |-- n_replies: integer (nullable = true)
 |-- picture: string (nullable = true)

+--------------------+--------------------+--------------+---------+--------------------+
|                 URL|                name|specialization|n_replies|             picture|
+--------------------+--------------------+--------------+---------+--------------------+
|https://www.dica3...|Dott. ALESSANDRO ...|          null|       83|/9j/4AAQSkZJRgABA...|
|https://www.dica3...|Dott. MICHELE AMENTA|          null|        1|/9j/4AAQSkZJRgABA...|
|https://www.dica3...|Dott. DOMENICO AN...|          null|        2|/9j/4AAQSkZJRgABA...|
|https://www.dica3...| Dott. FRANCO ASTORE|          null|        2|/9j/4AAQSkZJRgABA...|
|https://www.dica3...|Prof. GIOVANNI EN...|          null|        2|/9j/4AAQSkZJRgABA...|
|https://www.dica3...|Dott. FRANCESCO A...|   

In [10]:
dica33_doctors_df = load_dataframe(spark, '../dica33_doctors/data')
dica33_doctors_df = dica33_doctors_df.drop_duplicates(['URL'])
dica33_doctors_df = dica33_doctors_df.na.drop(how='any', subset=['name', 'specialization'])
dica33_doctors_df = dica33_doctors_df.withColumn('specialization', mapping_udf(dica33_doctors_df['specialization']))
dica33_doctors_df = dica33_doctors_df.where(~col('specialization').contains('MISSING'))
#replace default image with null
default_picture = dica33_doctors_df.groupBy('picture').count().where(col('count')>1).limit(1).collect()[0]['picture']
dica33_doctors_df = dica33_doctors_df.replace(default_picture, None, ['picture'])

../dica33_doctors/data loaded
root
 |-- URL: string (nullable = true)
 |-- name: string (nullable = true)
 |-- specialization: string (nullable = true)
 |-- n_replies: integer (nullable = true)
 |-- picture: string (nullable = true)



In [17]:
dica33_doctors_df.write.json('../dica33_doctors/json_dica33_doctors/doctors', mode="overwrite")